In [1]:
%pip install -q sec-api

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
from sec_api import XbrlApi


In [3]:
API_KEY = 'd6f32483c70c26c69ef8cae4010108a9667647438edd7c4d9876157d2ec3b82c'

In [4]:

xbrlApi = XbrlApi(API_KEY)

In [34]:
# URL of Microsoft's 10-K filings
url_10k_msft = 'https://www.sec.gov/ix?doc=/Archives/edgar/data/0000789019/000095017024087843/msft-20240630.htm'
url_10k_msft_2023 = 'https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm'
url_10k_aapl = 'https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000032019324000123/aapl-20240928.htm'
url_10k_aapl_2023 = 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm'
url_10k_tsla = 'https://www.sec.gov/ix?doc=/Archives/edgar/data/0001318605/000162828025003063/tsla-20241231.htm'
url_10k_tsla_2023 = 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm'

# Convert the XBRL data to JSON format
msft_xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k_msft)
url_10k_msft_2023_xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k_msft_2023)
aapl_xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k_aapl)
aapl_xbrl_json_2023 = xbrlApi.xbrl_to_json(htm_url=url_10k_aapl_2023)
tsla_xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k_tsla)
tsla_xbrl_json_2023 = xbrlApi.xbrl_to_json(htm_url=url_10k_tsla_2023)

In [6]:
print("Keys of income statement dictionary in XBRL from Microsoft's 10-K filing")
print('--------------------------------------------------------------------')
print(*list(msft_xbrl_json['StatementsOfIncome'].keys()), sep='\n')

Keys of income statement dictionary in XBRL from Microsoft's 10-K filing
--------------------------------------------------------------------
RevenueFromContractWithCustomerExcludingAssessedTax
CostOfGoodsAndServicesSold
GrossProfit
ResearchAndDevelopmentExpense
SellingAndMarketingExpense
GeneralAndAdministrativeExpense
OperatingIncomeLoss
NonoperatingIncomeExpense
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest
IncomeTaxExpenseBenefit
NetIncomeLoss
EarningsPerShareBasic
EarningsPerShareDiluted
WeightedAverageNumberOfSharesOutstandingBasic
WeightedAverageNumberOfDilutedSharesOutstanding


In [ ]:

# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '_' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 


income_statement_msft = get_income_statement(msft_xbrl_json)
income_statement_aapl = get_income_statement(aapl_xbrl_json)
income_statement_tsla = get_income_statement(tsla_xbrl_json)

income_statement_msft = income_statement_msft.loc[['RevenueFromContractWithCustomerExcludingAssessedTax', 'NetIncomeLoss']]
income_statement_aapl = income_statement_aapl.loc[['RevenueFromContractWithCustomerExcludingAssessedTax', 'NetIncomeLoss']]
income_statement_tsla = income_statement_tsla.loc[['RevenueFromContractWithCustomerExcludingAssessedTax', 'NetIncomeLoss']]

# reset index and name it
income_statement_msft = income_statement_msft.reset_index().rename(columns={'index': 'Variables'})
income_statement_aapl = income_statement_aapl.reset_index().rename(columns={'index': 'Variables'})
income_statement_tsla = income_statement_tsla.reset_index().rename(columns={'index': 'Variables'})

# pivot 
income_statement_msft_melt = income_statement_msft.melt(id_vars=["Variables"], var_name="Date", value_name="Value")
income_statement_aapl_melt = income_statement_aapl.melt(id_vars=["Variables"], var_name="Date", value_name="Value")
income_statement_tsla_melt = income_statement_tsla.melt(id_vars=["Variables"], var_name="Date", value_name="Value")

# Keep only last part of Date string column (i.e., keep only the date of the filing)
income_statement_msft_melt['Date'] = income_statement_msft_melt['Date'].apply(lambda x: x.split('_')[1])
income_statement_aapl_melt['Date'] = income_statement_aapl_melt['Date'].apply(lambda x: x.split('_')[1])
income_statement_tsla_melt['Date'] = income_statement_tsla_melt['Date'].apply(lambda x: x.split('_')[1])

print("Income statement of Microsoft's 2023 10-K filing as dataframe")
print('---------------------------------------------------------')
income_statement_msft_melt

Income statement of Microsoft's 2023 10-K filing as dataframe
---------------------------------------------------------


,Variables,Date,Value
0,RevenueFromContractWithCustomerExcludingAssess...,2024-06-30,245122000000
1,NetIncomeLoss,2024-06-30,88136000000
2,RevenueFromContractWithCustomerExcludingAssess...,2023-06-30,211915000000
3,NetIncomeLoss,2023-06-30,72361000000
4,RevenueFromContractWithCustomerExcludingAssess...,2022-06-30,198270000000
5,NetIncomeLoss,2022-06-30,72738000000


In [8]:
print("Keys of balance sheets dictionary in XBRL from Microsoft's 10-K filing")
print('--------------------------------------------------------------------')
print(*list(msft_xbrl_json['BalanceSheets'].keys()), sep='\n')

Keys of balance sheets dictionary in XBRL from Microsoft's 10-K filing
--------------------------------------------------------------------
CashAndCashEquivalentsAtCarryingValue
ShortTermInvestments
CashCashEquivalentsAndShortTermInvestments
AccountsReceivableNetCurrent
InventoryNet
OtherAssetsCurrent
AssetsCurrent
PropertyPlantAndEquipmentNet
OperatingLeaseRightOfUseAsset
LongTermInvestments
Goodwill
FiniteLivedIntangibleAssetsNet
OtherAssetsNoncurrent
Assets
AccountsPayableCurrent
CommercialPaper
LongTermDebtCurrent
EmployeeRelatedLiabilitiesCurrent
AccruedIncomeTaxesCurrent
ContractWithCustomerLiabilityCurrent
OtherLiabilitiesCurrent
LiabilitiesCurrent
LongTermDebtNoncurrent
AccruedIncomeTaxesNoncurrent
ContractWithCustomerLiabilityNoncurrent
DeferredIncomeTaxLiabilitiesNet
OperatingLeaseLiabilityNoncurrent
OtherLiabilitiesNoncurrent
Liabilities
CommitmentsAndContingencies
CommonStocksIncludingAdditionalPaidInCapital
RetainedEarningsAccumulatedDeficit
AccumulatedOtherComprehensiveIn

In [ ]:
# convert XBRL-JSON of Balance sheets to pandas dataframe
def get_balance_sheets(xbrl_json):
    balance_sheets_store = {}

    # iterate over each US GAAP item in the balance sheet statement
    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact and 'value' in fact:
                period = fact['period']
                
                # Handle duration vs instant periods
                if 'startDate' in period and 'endDate' in period:
                    index = f"{period['startDate']}-{period['endDate']}"
                elif 'instant' in period:
                    index = period['instant']
                else:
                    continue  # skip if period structure is unexpected
                # index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)
                        

        balance_sheets_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheets = pd.DataFrame(balance_sheets_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return balance_sheets.T 

# List of GAAP items to keep
list_gaap_items = ['Assets', 'Liabilities']

# get balance sheets for each company
balance_sheets_msft = get_balance_sheets(msft_xbrl_json)
balance_sheets_msft_2023 = get_balance_sheets(url_10k_msft_2023_xbrl_json)
balance_sheets_aapl = get_balance_sheets(aapl_xbrl_json)
balance_sheets_aapl_2023 = get_balance_sheets(aapl_xbrl_json_2023)
balance_sheets_tsla = get_balance_sheets(tsla_xbrl_json)
balance_sheets_tsla_2023 = get_balance_sheets(tsla_xbrl_json_2023)

# join 2024 and 2023 dataframes for each company
balance_sheets_msft = pd.concat([balance_sheets_msft, balance_sheets_msft_2023], axis=1)
balance_sheets_aapl = pd.concat([balance_sheets_aapl, balance_sheets_aapl_2023], axis=1)    
balance_sheets_tsla = pd.concat([balance_sheets_tsla, balance_sheets_tsla_2023], axis=1)

# select only Assets and Liabilities
balance_sheets_msft = balance_sheets_msft.loc[list_gaap_items]
balance_sheets_aapl = balance_sheets_aapl.loc[list_gaap_items]
balance_sheets_tsla = balance_sheets_tsla.loc[list_gaap_items]

# drop columns with null values if any
balance_sheets_msft = balance_sheets_msft.dropna(axis=1, how='all')
balance_sheets_aapl = balance_sheets_aapl.dropna(axis=1, how='all')
balance_sheets_tsla = balance_sheets_tsla.dropna(axis=1, how='all')

# drop duplicate columns if any
balance_sheets_msft = balance_sheets_msft.loc[:,~balance_sheets_msft.columns.duplicated()]
balance_sheets_aapl = balance_sheets_aapl.loc[:,~balance_sheets_aapl.columns.duplicated()]
balance_sheets_tsla = balance_sheets_tsla.loc[:,~balance_sheets_tsla.columns.duplicated()]

# reset index and name it
balance_sheets_msft = balance_sheets_msft.reset_index().rename(columns={'index': 'Variables'})
balance_sheets_aapl = balance_sheets_aapl.reset_index().rename(columns={'index': 'Variables'})
balance_sheets_tsla = balance_sheets_tsla.reset_index().rename(columns={'index': 'Variables'})

# pivot 
balance_sheets_msft_melt = balance_sheets_msft.melt(id_vars=["Variables"], var_name="Date", value_name="Value")
balance_sheets_aapl_melt = balance_sheets_aapl.melt(id_vars=["Variables"], var_name="Date", value_name="Value")
balance_sheets_tsla_melt = balance_sheets_tsla.melt(id_vars=["Variables"], var_name="Date", value_name="Value")

print("Balance Sheets of Microsoft's 2023 10-K filing as dataframe")
print('---------------------------------------------------------')
balance_sheets_msft_melt

Balance Sheets of Microsoft's 2023 10-K filing as dataframe
---------------------------------------------------------


,Variables,Date,Value
0,Assets,2023-06-30,411976000000
1,Liabilities,2023-06-30,205753000000
2,Assets,2024-06-30,512163000000
3,Liabilities,2024-06-30,243686000000
4,Assets,2022-06-30,364840000000
5,Liabilities,2022-06-30,198298000000


In [49]:
def get_cashflow(xbrl_json):
    cashflow_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact and 'value' in fact:
                # ensure no index duplicates are created
                period = fact['period']
                
                # Handle duration vs instant periods
                if 'startDate' in period and 'endDate' in period:
                    index = f"{period['startDate']}-{period['endDate']}"
                elif 'instant' in period:
                    index = period['instant']
                else:
                    continue  # skip if period structure is unexpected

                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        cashflow_store[usGaapItem] = pd.Series(values, index=indicies) 

    cashflow = pd.DataFrame(cashflow_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return cashflow.T 


cashflow_msft = get_cashflow(msft_xbrl_json)
cashflow_aapl = get_cashflow(aapl_xbrl_json)
cashflow_tsla = get_cashflow(tsla_xbrl_json)

# cashflow_msft = cashflow_msft.loc[['Assets', 'Liabilities']]
# cashflow_aapl = cashflow_aapl.loc[['Assets', 'Liabilities']]
# cashflow_tsla = cashflow_tsla.loc[['Assets', 'Liabilities']]

# # reset index and name it
# cashflow_msft = cashflow_msft.reset_index().rename(columns={'index': 'Variables'})
# cashflow_aapl = cashflow_aapl.reset_index().rename(columns={'index': 'Variables'})
# cashflow_tsla = cashflow_tsla.reset_index().rename(columns={'index': 'Variables'})

# # pivot 
# cashflow_msft_melt = cashflow_msft.melt(id_vars=["Variables"], var_name="Date", value_name="Value")
# cashflow_aapl_melt = cashflow_aapl.melt(id_vars=["Variables"], var_name="Date", value_name="Value")
# cashflow_tsla_melt = cashflow_tsla.melt(id_vars=["Variables"], var_name="Date", value_name="Value")

print("Cash Flow of Microsoft's 2023 10-K filing as dataframe")
print('---------------------------------------------------------')
cashflow_msft

Cash Flow of Microsoft's 2023 10-K filing as dataframe
---------------------------------------------------------


,2022-06-30,2023-06-30,2024-06-30
CashAndCashEquivalentsAtCarryingValue,NaN,34704000000,18315000000
ShortTermInvestments,NaN,76558000000,57228000000
CashCashEquivalentsAndShortTermInvestments,NaN,111262000000,75543000000
AccountsReceivableNetCurrent,NaN,48688000000,56924000000
InventoryNet,NaN,2500000000,1246000000
OtherAssetsCurrent,NaN,21807000000,26021000000
AssetsCurrent,NaN,184257000000,159734000000
PropertyPlantAndEquipmentNet,NaN,95641000000,135591000000
OperatingLeaseRightOfUseAsset,NaN,14346000000,18961000000
LongTermInvestments,NaN,9879000000,14600000000
